## 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 저장소 클론 (기존 디렉토리 있으면 삭제 후 재클론)
!rm -rf -prompt-firewall
!git clone https://github.com/mlnyx/-prompt-firewall.git
%cd ./-prompt-firewall

In [ ]:
# 필수 패키지 설치
!pip install -q transformers torch sentence-transformers pyyaml pandas tqdm accelerate bitsandbytes

## 2. Hugging Face 토큰 설정 (Llama 3 접근용)

1. https://huggingface.co/settings/tokens 에서 토큰 생성
2. https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct 에서 모델 접근 권한 요청
3. 아래 셀에 토큰 입력

In [ ]:
from huggingface_hub import login

# 여기에 토큰 입력
HF_TOKEN = "hf_..."
login(token=HF_TOKEN)

## 3. 평가 실행

**평가 옵션:**
- **전체 파이프라인**: Stage 1-2-3 순차 실행
- **개별 Stage**: 각 Stage를 독립적으로 실행하여 성능 비교

**목적:** 각 Stage의 효율성을 검증하고, 초기 단계에서 쉬운 공격을 적은 자원으로 차단하는지 확인

In [ ]:
### 3-1. 전체 파이프라인 (Stage 1-2-3)

# Stage 1-2-3 전체 실행
!python evaluate.py

In [ ]:
# Stage 1-2 실행 (ML 모델 추가)
from tester_framework.runners import Stage2LocalRunner

print("\n" + "=" * 60)
print("Stage 1-2 평가: 규칙 + ML 앙상블")
print("=" * 60)

# Stage 1 ESCALATE 항목만 Stage 2로
escalated = [s for s in results if s.s1_decision == 'ESCALATE']
print(f"\nStage 2 입력: {len(escalated)}개")

if escalated:
    pop2 = Population(seeds=escalated)
    runner2 = Stage2LocalRunner()
    tester2 = Tester(pop2, runner2)
    results2 = tester2.run_all()
    
    # 결과 요약
    allow2 = sum(1 for s in results2 if s.s2_decision == 'ALLOW')
    block2 = sum(1 for s in results2 if s.s2_decision == 'BLOCK')
    rewrite = sum(1 for s in results2 if s.s2_decision == 'REWRITE')
    
    print(f"\n[Stage 2 결과]")
    print(f"  ALLOW: {allow2}개 ({allow2/len(results2)*100:.1f}%)")
    print(f"  BLOCK: {block2}개 ({block2/len(results2)*100:.1f}%)")
    print(f"  REWRITE: {rewrite}개 ({rewrite/len(results2)*100:.1f}%) → Stage 3 필요")
    
    print(f"\n[전체 처리 현황]")
    print(f"  Stage 1 처리: {allow + block}개")
    print(f"  Stage 2 처리: {allow2 + block2}개")
    print(f"  Stage 3 필요: {rewrite}개")
    print(f"  → Stage 1-2만으로 {(allow+block+allow2+block2)/len(results)*100:.1f}% 처리 완료")

In [ ]:
# Stage 1만 실행 (규칙 기반 필터)
from tester_framework.core import Population
from tester_framework.runners import Stage1LocalRunner
from tester_framework.orchestrator import Tester
from prompt_firewall.utils.config import TEST_DATA_PATH

print("=" * 60)
print("Stage 1 단독 평가: 규칙 기반 필터")
print("=" * 60)

pop = Population()
pop.create_population_from_file(TEST_DATA_PATH)

runner = Stage1LocalRunner()
tester = Tester(pop, runner)
results = tester.run_all()

# 결과 요약
allow = sum(1 for s in results if s.s1_decision == 'ALLOW')
block = sum(1 for s in results if s.s1_decision == 'BLOCK')
escalate = sum(1 for s in results if s.s1_decision == 'ESCALATE')

print(f"\n[결과]")
print(f"  ALLOW: {allow}개 ({allow/len(results)*100:.1f}%)")
print(f"  BLOCK: {block}개 ({block/len(results)*100:.1f}%)")
print(f"  ESCALATE: {escalate}개 ({escalate/len(results)*100:.1f}%) → Stage 2 필요")

### 3-2. Stage별 개별 실행 (성능 비교용)

각 Stage를 독립적으로 실행하여:
- Stage 1만으로 얼마나 차단 가능한지
- Stage 2 추가 시 정확도 향상
- 각 Stage의 처리 시간 및 자원 사용량 측정

### 3-3. Stage 2 결과 샘플링 (Stage 3 준비)

In [ ]:
import pandas as pd

# Stage 2 결과 로드
jailbreak_df = pd.read_csv('data/stage2_rewrites(jailbreak).txt')
benign_df = pd.read_csv('data/stage2_rewrites(benign).txt')

print(f"Jailbreak: {len(jailbreak_df)}개")
print(f"Benign: {len(benign_df)}개")

# Jailbreak: score 낮은 순 (애매한 악성)
jailbreak_samples = jailbreak_df.nsmallest(256, 'score')

# Benign: score 높은 순 (애매한 정상)
benign_samples = benign_df.nlargest(256, 'score')

# 샘플 결합
samples = pd.concat([jailbreak_samples, benign_samples])

# 샘플 저장
samples.to_csv('stage3_samples.csv', index=False)
print(f"\n✓ Stage 3 샘플 512개 저장: stage3_samples.csv")
print(f"  - Jailbreak (low score): {len(jailbreak_samples)}개")
print(f"  - Benign (high score): {len(benign_samples)}개")

In [ ]:
# Stage 3 평가 (Llama 3 필요)
# Llama 3 접근 승인 완료 후 주석 해제하여 실행

# from tester_framework.runners import Stage3LocalRunner
# 
# print("\n" + "=" * 60)
# print("Stage 3 평가: LLM 재작성")
# print("=" * 60)
# 
# pop3 = Population()
# pop3.create_population_from_file('stage3_samples.csv')
# 
# runner3 = Stage3LocalRunner(use_local_llm=True)
# tester3 = Tester(pop3, runner3)
# results3 = tester3.run_all()
# 
# # 결과 저장
# pop3.save_to_csv('stage3_results.csv')
# print(f'\n✓ 완료: {len(results3)}개')

In [ ]:
## 5. 결과 다운로드

In [ ]:
from google.colab import files

# Stage 2 결과 다운로드
!ls -lh stage2_rewrites.txt && files.download('stage2_rewrites.txt')

# Stage 2 점수 다운로드
!ls -lh data/s2_all_scores.csv && files.download('data/s2_all_scores.csv')